In [0]:
%sql  
/* join_silver_feedback_enriched.ipynb

SOURCE: Snapshot join of Silver feedback and current provider attributes
OUTPUT: kardia_silver.silver_feedback_enriched

PATTERN: LEFT JOIN from feedback to SCD‑2 providers, keeping only the current provider row
TRIGGER: CTAS; rerun after any update to feedback or providers

NOTE:
 - LEFT JOIN preserves all feedback rows
 - IS_CURRENT = TRUE ensures only the current provider info is used
*/

-- Ensure Silver DB exists
CREATE DATABASE IF NOT EXISTS kardia_silver;
USE kardia_silver;

In [0]:
%sql
-- 2. Join feedback to current provider attributes (no CTE – inline subquery)
CREATE OR REPLACE TABLE silver_feedback_enriched AS
WITH current_providers AS (
  SELECT provider_id,
         provider_specialty,
         provider_location
  FROM   kardia_silver.silver_providers
  WHERE  is_current = TRUE
)
SELECT
    f.feedback_id,
    f.provider_id,
    cp.provider_specialty,
    cp.provider_location,
    f.timestamp,
    f.visit_id,
    f.satisfaction_score,
    f.comments,
    f.source,
    f.tags,
    f.metadata,
    f._ingest_ts,
    f._source_file,
    f._batch_id
FROM kardia_silver.silver_feedback f
LEFT JOIN current_providers cp
  ON f.provider_id = cp.provider_id;

In [0]:
%sql
-- 3. Preview: Most recent 5 feedback records
SELECT *
FROM kardia_silver.silver_feedback_enriched
ORDER BY timestamp DESC
LIMIT 5;